# Lab 2: ?

Lab session by:
* Daniel Hess
* Pandelis Laurens Symeonidis

# Imports

In [15]:
import nltk
import pandas as pd
from nltk.metrics.distance import jaccard_distance
from scipy.stats import pearsonr


-there are some shortcomings in this approach, for instance if we consider the tokens the uppcercase version and lowercase version will be two diff tokens so we can check if we are getting better results with everyhting lowercase or not

- what does stopword mean? Its a word with no meaning, if we add words with no meaning with same wiehghts as rest of the tokens then the metric can be poisoned, so we can try to remove the stopwords (include this in analysis)

In [21]:
dt = pd.read_csv('./STS.input.SMTeuroparl.txt',sep='\t',header=None)
nltk.download('punkt_tab')

dt_tokenized = dt.map(nltk.word_tokenize)
dt_tokenized.head()

jaccard_distances = dt_tokenized.apply(lambda row: jaccard_distance(set(row[0]), set(row[1])), axis=1)
print(jaccard_distances.head())


similarity_scores = 1 - jaccard_distances
print(similarity_scores.shape)

gold_standard_dt = pd.read_csv('./STS.gs.SMTeuroparl.txt',sep='\t',header=None)

comparison = pearsonr(similarity_scores, gold_standard_dt[0])
print(comparison[0])



0    0.653846
1    0.214286
2    0.608696
3    0.454545
4    0.000000
dtype: float64
(459,)
0.4504977169318686


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/pandelissymeonidis/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# Analysis